In [ ]:
import torchio as tio
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from nibabel.processing import conform
import seaborn as sns
from statannotations.Annotator import Annotator
from scipy import ndimage

In [ ]:
factor=[[0.735 , 0.735],
        [0.775 , 0.775],
        [0.837 , 0.837],
        [0.895, 0.895],
        [0.95 , 0.95],
        [0.96 , 0.96],
        [0.97 , 0.97], 
        [0.98 , 0.98],
        [0.99 , 0.99],
        [1 , 1],
        [1.01 , 1.01],
        [1.02 , 1.02],
        [1.0298 , 1.0298],
        [1.0394 , 1.0394],
        [1.049 , 1.049],
        [1.0958 , 1.0958],
        [1.15 , 1.15],
        [1.21 , 1.21]]
teoric = []
for i in range (len(factor)):
    teoric.append(factor[i][1]**2*100)
teoric[4]

In [ ]:
df=[]

In [ ]:
def CenterANDCrop (image_SC_soft_pad,image_SC_unc_pad,image_GM_soft_pad,image_GM_unc_pad,image_T2_pad):
    for j in range(1):
        x,y=ndimage.measurements.center_of_mass(image_SC_soft_pad[:,:,j]) #Definiton de center of mass pour SC, et on va utilisser le meme centre pour GM et T2
        x,y=int(round(x)),int(round(y)) 
        xmin = x - int(Size_X/2)
        xmax = x + int(Size_X/2)
        ymin = y - int(Size_Y/2)
        ymax = y + int(Size_Y/2)
        
        img_new_soft_SC[:,:,j]=image_SC_soft_pad[xmin:xmax,ymin:ymax,j]
        img_new_unc_SC[:,:,j]=image_SC_unc_pad[xmin:xmax,ymin:ymax,j]
        img_new_soft_GM[:,:,j]=image_GM_soft_pad[xmin:xmax,ymin:ymax,j]
        img_new_unc_GM[:,:,j]=image_GM_unc_pad[xmin:xmax,ymin:ymax,j]
        img_new_T2[:,:,j]=image_T2_pad[xmin:xmax,ymin:ymax,j]
        
    SC_soft = nib.Nifti1Image(img_new_soft_SC, image_data_soft_SC.affine, header_info_soft_SC)
    GM_soft = nib.Nifti1Image(img_new_soft_GM, image_data_soft_SC.affine, header_info_soft_SC)
    SC_unc = nib.Nifti1Image(img_new_unc_SC, image_data_soft_SC.affine, header_info_soft_SC)
    GM_unc = nib.Nifti1Image(img_new_unc_GM, image_data_soft_SC.affine, header_info_soft_SC)
    T2 = nib.Nifti1Image(img_new_T2, image_data_soft_SC.affine, header_info_soft_SC)
    path_database_out = '/mnt/d/crmbm/pro_scaling_3t/centering_last/'
    nib.save(SC_soft,path_database_out + 'soft_sc_'+ str(slices).zfill(4)+'_'+ str(transf).zfill(2)+ '_center.nii.gz')
    nib.save(GM_soft,path_database_out + 'soft_gm_'+ str(slices).zfill(4)+'_'+ str(transf).zfill(2)+ '_center.nii.gz')
    nib.save(SC_unc,path_database_out + 'unc_sc_'+ str(slices).zfill(4)+'_'+ str(transf).zfill(2)+ '_center.nii.gz')
    nib.save(GM_unc,path_database_out + 'unc_gm_'+ str(slices).zfill(4)+'_'+ str(transf).zfill(2)+ '_center.nii.gz')
    nib.save(T2,path_database_out + 't2s_'+ str(slices).zfill(4)+'_'+ str(transf).zfill(2)+ '_center.nii.gz')
    

In [ ]:
for slices in range (11):
    for transf in range (18):
        grade_y=factor[transf][1]
        grade_x=factor[transf][0]
        path_database = '/mnt/d/crmbm/pro_scaling_3t/3T/'
        image = tio.ScalarImage(path_database + 't2s_'+ str(slices).zfill(4)+ '.nii.gz')
        label_soft_gm = tio.LabelMap(path_database +  'soft_gm_'+ str(slices).zfill(4)+ '.nii.gz')
        label_soft_sc = tio.LabelMap(path_database +  'soft_sc_'+ str(slices).zfill(4)+ '.nii.gz')
        label_unc_gm = tio.LabelMap(path_database + 'unc_gm_'+ str(slices).zfill(4)+ '.nii.gz')
        label_unc_sc = tio.LabelMap(path_database + 'unc_sc_'+ str(slices).zfill(4)+ '.nii.gz')
        Subjects={"image":image,"label_soft_gm":label_soft_gm,"label_soft_sc":label_soft_sc,
                  "label_unc_gm":label_unc_gm,"label_unc_sc":label_unc_sc}
        Subjects=tio.Subject(Subjects)
        transforms = [tio.RandomAffine(scales=(grade_x,grade_x,grade_y,grade_y,1,1),degrees=(0),isotropic = False,
                                       center= 'origin',image_interpolation='linear',label_interpolation='linear')]
        transform = tio.Compose(transforms)
        transformed_Aff = transform(Subjects)
        transformed_Aff.plot() 

        path_database = '/mnt/d/crmbm/pro_scaling_3t/scaling_last/'
        save_image =(path_database +  't2s_'+ str(slices).zfill(4)+'_'+ str(transf).zfill(2)+ '.nii.gz')
        (transformed_Aff["image"]).save(save_image)
        save_soft_gm =(path_database  +  'soft_gm_'+ str(slices).zfill(4)+'_'+ str(transf).zfill(2)+ '.nii.gz')
        (transformed_Aff["label_soft_gm"]).save(save_soft_gm)
        save_soft_sc =(path_database  +  'soft_sc_'+ str(slices).zfill(4)+'_'+ str(transf).zfill(2)+ '.nii.gz')
        (transformed_Aff["label_soft_sc"]).save(save_soft_sc)         
        save_unc_gm =(path_database  +  'unc_gm_'+ str(slices).zfill(4)+'_'+ str(transf).zfill(2)+ '.nii.gz')
        (transformed_Aff["label_unc_gm"]).save(save_unc_gm)
        save_unc_sc =(path_database  +  'unc_sc_'+ str(slices).zfill(4)+'_'+ str(transf).zfill(2)+ '.nii.gz')
        (transformed_Aff["label_unc_sc"]).save(save_unc_sc)

        image_data_soft_SC = nib.load(path_database + 'soft_sc_'+ str(slices).zfill(4)+'_'+ str(transf).zfill(2)+ '.nii.gz')
        header_info_soft_SC = image_data_soft_SC.header
        image_soft_SC = np.array(image_data_soft_SC.get_fdata())
        image_data_soft_GM = nib.load(path_database + 'soft_gm_'+ str(slices).zfill(4)+'_'+ str(transf).zfill(2)+ '.nii.gz')
        image_soft_GM = np.array(image_data_soft_GM.get_fdata())

        image_data_unc_GM = nib.load(path_database + 'unc_gm_'+ str(slices).zfill(4)+'_'+ str(transf).zfill(2)+ '.nii.gz')
        image_unc_GM = np.array(image_data_unc_GM.get_fdata())
        image_data_unc_SC = nib.load(path_database + 'unc_sc_'+ str(slices).zfill(4)+'_'+ str(transf).zfill(2)+ '.nii.gz')
        image_unc_SC = np.array(image_data_unc_SC.get_fdata())

        image_data_t2s = nib.load(path_database + 't2s_'+ str(slices).zfill(4)+'_'+ str(transf).zfill(2)+ '.nii.gz')
        image_t2s = np.array(image_data_t2s.get_fdata())

        Size_X=(image_t2s.shape)[0]+600
        Size_Y=(image_t2s.shape)[1]+600
        Size_Z=(image_t2s.shape)[2]
        image_SC_soft_pad=np.empty((Size_X,Size_Y,Size_Z))
        image_SC_unc_pad=np.empty((Size_X,Size_Y,Size_Z))
        image_GM_soft_pad=np.empty((Size_X,Size_Y,Size_Z))
        image_GM_unc_pad=np.empty((Size_X,Size_Y,Size_Z))
        image_T2_pad=np.empty((Size_X,Size_Y,Size_Z))
        for i in range((image_t2s.shape)[2]):
                image_SC_soft_pad[:,:,i]=np.pad(image_soft_SC[:,:,i], 300, 'constant')
                image_GM_soft_pad[:,:,i]=np.pad(image_soft_GM[:,:,i], 300, 'constant')
                image_SC_unc_pad[:,:,i]=np.pad(image_unc_SC[:,:,i], 300, 'constant')
                image_GM_unc_pad[:,:,i]=np.pad(image_unc_GM[:,:,i], 300, 'constant')
                image_T2_pad[:,:,i]=np.pad(image_t2s[:,:,i], 300, 'constant')
        Size_X=210
        Size_Y=210
        Size_Z=(image_t2s.shape)[2]
        img_new_soft_SC=np.empty((Size_X,Size_Y,Size_Z))
        img_new_unc_SC=np.empty((Size_X,Size_Y,Size_Z))
        img_new_soft_GM=np.empty((Size_X,Size_Y,Size_Z))
        img_new_unc_GM=np.empty((Size_X,Size_Y,Size_Z))
        img_new_T2=np.empty((Size_X,Size_Y,Size_Z))

        CenterANDCrop(image_SC_soft_pad,image_SC_unc_pad,image_GM_soft_pad,image_GM_unc_pad,image_T2_pad)